<a href="https://colab.research.google.com/github/tylergusmyers/ML_token_prices/blob/master_som/notebook/btc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the required libraries
from IPython.display import clear_output
try:
  #!pip install pystan
  !pip install pytrends
  !pip install yfinance
  !pip install yahoofinancials
except:
  print("Error installing libraries")
finally:
  clear_output()
  print('Libraries successfully installed')

Libraries successfully installed


In [9]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from pathlib import Path
import yfinance as yf


import yfinance as yf
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from pytrends.request import TrendReq


PrePare the data to be used

In [10]:
#pytrends = TrendReq(hl=’en-US’, tz=360)
pytrends = TrendReq()

In [22]:
pytrends.build_payload(kw_list=['bitcoin', 'crypto', 'cryptocurrency', 'btc', 'Block Chain'], timeframe='2017-01-01 2022-01-12')

# Interest

df = pytrends.interest_over_time()

df

,bitcoin,crypto,cryptocurrency,btc,Block Chain,isPartial
date,,,,,,
2017-01-01,6,0,0,1,0,False
2017-01-08,4,0,0,1,0,False
2017-01-15,3,0,0,1,0,False
2017-01-22,3,0,0,1,0,False
2017-01-29,3,0,0,1,0,False
...,...,...,...,...,...,...
2021-12-12,19,13,3,3,0,False
2021-12-19,17,12,2,3,0,False
2021-12-26,15,12,2,3,0,False


In [26]:
btc = yf.Ticker("BTC-USD")

# get historical market data
btc_usd_df = btc.history(
                          start='2017-01-01', 
                      end='2022-01-12', 
                      progress=False,
)


btc_usd_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008,0,0
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992,0,0
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000,0,0
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,344945984,0,0
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,0,0
...,...,...,...,...,...,...,...
2022-01-08,41561.464844,42228.941406,40672.277344,41733.941406,28066355845,0,0
2022-01-09,41734.726562,42663.949219,41338.160156,41911.601562,21294384372,0,0
2022-01-10,41910.230469,42199.484375,39796.570312,41821.261719,32104232331,0,0


In [118]:
dates_df = pd.DataFrame(
    {'Date':pd.date_range('2017-01-01', '2022-01-12')}
)


#dates_df.set_index('date')
#dates_df.rename(columns={'date': 'holiday'})
dates_df['holiday'] = dates_df['Date']

dates_df = dates_df.set_index('Date')
dates_df

,holiday
Date,
2017-01-01,2017-01-01
2017-01-02,2017-01-02
2017-01-03,2017-01-03
2017-01-04,2017-01-04
2017-01-05,2017-01-05
...,...
2022-01-08,2022-01-08
2022-01-09,2022-01-09
2022-01-10,2022-01-10


In [120]:
cal = calendar()

holidays = cal.holidays(
    start=dates['date'].min(),
    end=dates['date'].max()
)

holidays

DatetimeIndex(['2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29',
               '2017-07-04', '2017-09-04', '2017-10-09', '2017-11-10',
               '2017-11-23', '2017-12-25', '2018-01-01', '2018-01-15',
               '2018-02-19', '2018-05-28', '2018-07-04', '2018-09-03',
               '2018-10-08', '2018-11-12', '2018-11-22', '2018-12-25',
               '2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27',
               '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11',
               '2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20',
               '2020-02-17', '2020-05-25', '2020-07-03', '2020-09-07',
               '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25',
               '2021-01-01', '2021-01-18', '2021-02-15', '2021-05-31',
               '2021-07-05', '2021-09-06', '2021-10-11', '2021-11-11',
               '2021-11-25', '2021-12-24', '2021-12-31'],
              dtype='datetime64[ns]', freq=None)

In [121]:
dates_df['holiday'] = dates_df['holiday'].isin(holidays).astype(int) 
dates_df

,holiday
Date,
2017-01-01,0
2017-01-02,1
2017-01-03,0
2017-01-04,0
2017-01-05,0
...,...
2022-01-08,0
2022-01-09,0
2022-01-10,0


In [127]:
btc_usd_df_01 = pd.concat ([btc_usd_df , dates_df], axis=1)

btc_usd_df_01

,Open,High,Low,Close,Volume,Dividends,Stock Splits,holiday
Date,,,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008,0,0,0
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992,0,0,1
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000,0,0,0
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,344945984,0,0,0
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,0,0,0
...,...,...,...,...,...,...,...,...
2022-01-08,41561.464844,42228.941406,40672.277344,41733.941406,28066355845,0,0,0
2022-01-09,41734.726562,42663.949219,41338.160156,41911.601562,21294384372,0,0,0
2022-01-10,41910.230469,42199.484375,39796.570312,41821.261719,32104232331,0,0,0


In [142]:
def train_test_split(btc_usd_df_01, test_size=0.2):
    split_row = len(btc_usd_df_01) - int(test_size * len(btc_usd_df_01))
    train_data = btc_usd_df_01.iloc[:split_row]
    test_data = btc_usd_df_01.iloc[split_row:]
    return train_data, test_data
train, test = train_test_split(hist, test_size=0.2)